# Fake address gen

In [5]:
import pandas as pd
import json
import re
import random
import string as string_lib

## Word list for fake address generation

Taken from <https://github.com/duyet/vietnamese-wordlist>, using the 11K words file.

In [35]:
with open("./InputData/Viet11K.txt", 'r', encoding="utf-8") as file:
    vn_words = file.read().splitlines()
vn_words

['a',
 'a dua',
 'a hoàn',
 'a phiến',
 'a tòng',
 'à',
 'ả',
 'ả đào',
 'ả giang hồ',
 'á',
 'á-căn-đình',
 'á khẩu',
 'á khôi',
 'á kim',
 'á rập',
 'ạ',
 'ác',
 'ác cảm',
 'ác chiến',
 'ác mộng',
 'ác nghiệt',
 'ác phụ',
 'ác tà',
 'ác tâm',
 'ác thú',
 'ách',
 'ạch',
 'ai',
 'ai ai',
 'ai điếu',
 'ai oán',
 'ải',
 'ải quan',
 'ái',
 'ái ân',
 'ái hữu',
 'ái khanh',
 'ái lực',
 'ái mộ',
 'ái nam ái nữ',
 'ái ngại',
 'ái nhĩ lan',
 'ái phi',
 'ái quốc',
 'ái tình',
 'am',
 'am hiểu',
 'am pe',
 'ảm đạm',
 'ám',
 'ám ảnh',
 'ám chỉ',
 'ám hại',
 'ám hiệu',
 'ám muội',
 'ám sát',
 'ám tả',
 'ám thị',
 'an',
 'an ba ni',
 'an bài',
 'an cư',
 'an dưỡng',
 'an nghỉ',
 'an ninh',
 'an phận',
 'an táng',
 'an tâm',
 'an toàn',
 'an ủi',
 'an vị',
 'án',
 'án mạng',
 'án ngữ',
 'án phí',
 'án sát',
 'án thư',
 'án tử hình',
 'ang',
 'ang áng',
 'áng',
 'anh',
 'anh ánh',
 'anh dũng',
 'anh đào',
 'anh em',
 'anh hùng',
 'anh linh',
 'anh tài',
 'anh thư',
 'anh tuấn',
 'ảnh',
 'ảnh ảo',
 'ả

## Import and adminstrative region data

In [6]:
with open("./InputData/region_tree.json", 'r') as file:
    region_tree = json.load(file)
    
with open("./InputData/reverse_index_dist_to_prov.json", 'r') as file:
    rev_index_dist_to_prov = json.load(file)
    
with open("./InputData/reverse_index_ward_to_prov.json", 'r') as file:
    rev_index_ward_to_prov = json.load(file)
    
with open("./InputData/reverse_index_ward_to_dist.json", 'r') as file:
    rev_index_ward_to_dist = json.load(file)

## String deterioration function

In [7]:
def expand_space_within_string(string: str, limit: (int, int), /, 
                               include_newline=False, newline_prob=0, 
                               include_tab=False, tab_prob=0) -> str:
    tokens = string.split()
    space_prob = 1 - (newline_prob if include_newline else 0) - (tab_prob if include_tab else 0)
    prob_weight = [space_prob, newline_prob, tab_prob]
    whitespace = [' ', '\n', '\t']
    result = str()
    
    for i, token in enumerate(tokens):
        result += token
        if i == len(tokens):
            pass
        num_space = random.randrange(*limit)
        spaces = ''.join(random.choices(whitespace, prob_weight, k=num_space))
        result += spaces
    
    return result

In [8]:
test = 'this is a test string'
expand = expand_space_within_string(
    test, (2, 8), 
    include_newline=True, newline_prob=0.1, 
    include_tab=True, tab_prob=0.05)
print(expand)

this
     is    a  test    
  string      


In [144]:
# by default, string will deteriorate by having their chars substituted
def substitute_string(string:str) -> str:
    position = random.randrange(0, len(string))
    replace_with = string_lib.ascii_letters + string_lib.digits + string_lib.punctuation
    
    return string[:position] + random.choice(replace_with) + string[position+1:]
    
    
def insert_string(string:str) -> str:
    position = random.randrange(0, len(string))
    insert_with = string_lib.ascii_letters + string_lib.digits + string_lib.punctuation
    
    return string[:position] + random.choice(insert_with) + string[position:]


def delete_string(string:str) -> str:
    position = random.randrange(0, len(string))
    
    return string[:position] + string[position+1:]

def transpose_string(string:str) -> str:
    if len(string) < 2:
        return string
    
    position = random.randrange(0, len(string) - 1)
    
    return string[:position] + string[position+1] + string[position] + string[position+2:]
    
# substitution always on by default    
def deteriorate_string(string:str, edit_distance=1, /,
                      allow_insertion=True, allow_deletion=True, allow_transposition=True, 
                      insert_prob=0.1, delete_prob=0.1, trans_prob=0.05) -> str:
    
    sub_prob = 1 - (insert_prob if allow_insertion else 0) \
        - (delete_prob if allow_deletion else 0) \
        - (trans_prob if allow_transposition else 0)
    action_weights = [sub_prob, insert_prob, delete_prob, trans_prob]
    actions = [substitute_string, insert_string, delete_string, transpose_string]
    
    for _ in range(edit_distance):
        action = random.choices(actions, action_weights, k=1)[0]
        string = action(string)
        
    return string
    

In [10]:
print(substitute_string('test'))
print(insert_string('test'))
print(delete_string('test'))
print(transpose_string('test'))

tbst
tpest
est
tset


In [13]:
deteriorate_string('this is a test string, it should be destroy incrementally', 6,
                  allow_insertion=True, allow_deletion=True, allow_transposition=True)

'tahis is a test string,>i` should eb destSoy 2ncrementally'

## Fake address generation

In [36]:
def get_prov_dist_ward_level_region(region_tr) -> {'province': str, 'district': str, 'ward': str}:
    random_prov_info = random.choice(list(region_tr.values()))
    random_dist_info = random.choice(list(random_prov_info['districts'].values()))
    random_ward_info = random.choice(list(random_dist_info['wards'].values()))
    
    return {
        'province': random_prov_info, 
        'district': random_dist_info, 
        'ward': random_ward_info
    }

In [37]:
get_prov_dist_ward_level_region(region_tree)

{'province': {'prefix': 'Tỉnh',
  'name': 'Sóc Trăng',
  'districts': {'Sóc Trăng': {'prefix': 'Thành phố',
    'name': 'Sóc Trăng',
    'wards': {'5': {'prefix': 'Phường', 'name': '5'},
     '7': {'prefix': 'Phường', 'name': '7'},
     '8': {'prefix': 'Phường', 'name': '8'},
     '6': {'prefix': 'Phường', 'name': '6'},
     '2': {'prefix': 'Phường', 'name': '2'},
     '4': {'prefix': 'Phường', 'name': '4'},
     '3': {'prefix': 'Phường', 'name': '3'},
     '1': {'prefix': 'Phường', 'name': '1'},
     '10': {'prefix': 'Phường', 'name': '10'}}},
   'Châu Thành': {'prefix': 'Huyện',
    'name': 'Châu Thành',
    'wards': {'Châu Thành': {'prefix': 'Thị trấn', 'name': 'Châu Thành'},
     'Hồ Đắc Kiện': {'prefix': 'Xã', 'name': 'Hồ Đắc Kiện'},
     'Phú Tâm': {'prefix': 'Xã', 'name': 'Phú Tâm'},
     'Thuận Hòa': {'prefix': 'Xã', 'name': 'Thuận Hòa'},
     'Phú Tân': {'prefix': 'Xã', 'name': 'Phú Tân'},
     'Thiện Mỹ': {'prefix': 'Xã', 'name': 'Thiện Mỹ'},
     'An Hiệp': {'prefix': 'Xã', 

In [129]:
def generate_house_address(with_sublevel=True, sublevel=2) -> str:
    num_list = [str(random.randrange(1, 200)) for _ in range(sublevel)]
    result = str()
    for i in range(sublevel):
        result += num_list[i]
        if i < sublevel-1:
            result += '/'
    
    return result


def generate_street_name(min_name_word_len=3, add_prefix=True) -> str:
    name = str()
    name_word_len = 0
    while name_word_len < min_name_word_len:
        words = random.choice(vn_words).split()
        word_len = len(words)
        name_word_len += word_len
        name += ' ' + ' '.join(word.capitalize() for word in words)
    
    name = name.strip()
    
    if add_prefix:
        return "Đường " + name
    else:
        return name

    
def generate_hamlet_name(min_name_word_len=2, add_prefix=True) -> str:
    prefixes = ["Thôn", "Xóm", "Ấp", "Đội"]
    
    if random.random() <= 0.5:
        name =  generate_street_name(min_name_word_len, add_prefix=False)
    else:
        name = str(random.randrange(1, 30))
    
    if add_prefix:
        return random.choice(prefixes) + ' ' + name
    else:
        return name

def generate_quarter_name(min_name_word_len=2, add_prefix=True) -> str:
    if random.random() <= 0.1:
        name =  generate_street_name(min_name_word_len, add_prefix=False)
    else:
        name = str(random.randrange(1, 30))
    
    if add_prefix:
        return "Khu phố" + ' ' + name
    else:
        return name
    
    
def generate_address_below_ward_level(add_house_address=True, # so nha
                                      add_street=True,        # ten duong
                                      add_hamlet=True,        # thon, xom, ap, doi
                                      add_quarter=True        # khu pho
                                     ) -> {
    'address': str,
    'street': str,
    'hamlet': str,
    'quarter': str
}:
    add_prefix_street = random.random() <= 0.5
    add_prefix_hamlet = random.random() <= 0.5
    add_prefix_quarter = random.random() <= 0.5
    
    return {
        'address': generate_house_address(),
        'street': generate_street_name(add_prefix=add_prefix_street),
        'hamlet': generate_hamlet_name(add_prefix=add_prefix_hamlet),
        'quarter': generate_quarter_name(add_prefix=add_prefix_quarter)
    }
    
    
def generate_address_from_ward_level_onward(add_ward=True, add_dist=True, add_prov=True) -> {
    'ward': str,
    'district': str,
    'province': str
}:
    selection = get_prov_dist_ward_level_region(region_tree)
    
    result = {    
        'ward': None,
        'district': None,
        'province': None
    }
    
    if add_ward:
        result['ward'] = (selection['ward']['prefix'] + ' ' if random.random() <= 0.5 else '') + \
            selection['ward']['name']
    if add_dist:
        result['district'] = (selection['district']['prefix'] + ' ' if random.random() <= 0.5 else '') + \
            selection['district']['name']
    if add_prov:
        result['province'] = (selection['province']['prefix'] + ' ' if random.random() <= 0.5 else '') + \
            selection['province']['name']
    
    return result
        

In [130]:
print(generate_house_address())
print(generate_street_name(min_name_word_len=3))
print(generate_hamlet_name(min_name_word_len=2))
print(generate_quarter_name(min_name_word_len=2))

77/54
Đường Cóp Sột Soạt
Ấp 10
Khu phố Viêm Xay


In [131]:
generate_address_below_ward_level()

{'address': '32/39',
 'street': 'Đường Thua Ủng Rắn Hổ Lửa',
 'hamlet': 'Ấp Thất Nhân Tâm',
 'quarter': '19'}

In [134]:
generate_address_from_ward_level_onward()

{'ward': 'Xã Vĩnh Hậu A',
 'district': 'Huyện Hoà Bình',
 'province': 'Tỉnh Bạc Liêu'}

In [139]:
def generate_fake_address(add_house_address=True, # so nha
                          add_street=True,        # ten duong
                          add_hamlet=True,        # thon, xom, ap, doi
                          add_quarter=True,       # khu pho
                          add_ward=True, add_dist=True, add_prov=True) -> str:
    
    below = generate_address_below_ward_level(
        add_house_address=add_house_address,
        add_street=add_street,
        add_hamlet=add_hamlet,
        add_quarter=add_quarter
    )
    above = generate_address_from_ward_level_onward(
        add_ward=add_ward, add_dist=add_dist, add_prov=add_prov
    )
    combined = {**below, **above}
    
    part_list = []
    for addr_part in combined.values():
        if addr_part is not None:
            part_list.append(addr_part)
    
    return ', '.join(part_list)

In [143]:
generate_fake_address()

'60/172, Chết Đuối Nhà Hàng, 22, Khu phố 12, Long Thới, Chợ Lách, Tỉnh Bến Tre'

In [154]:
for _ in range(20):
    param = {
        'add_house_address': random.random() <= 0.6,
        'add_street': random.random() <= 0.8,
        'add_hamlet': random.random() <= 0.5,
        'add_quarter': random.random() <= 0.5,
        'add_ward': random.random() <= 0.9,
        'add_dist': random.random() <= 0.95,
        'add_prov': random.random() <= 0.95,
    }
    addr = deteriorate_string(generate_fake_address(**param), 8)
    addr = expand_space_within_string(addr, (1, 3))
    print(addr)

63/]6, Như  hyầ  Tròd ;hrịa, Đội 29,  K4u  phố  1, Phường  4,  Trà  Vinh, Trà Vi!h  
180/5,, Thủ  Cômg Ngoài  Ra,  1X, Khu  phố 18,  Hậu  Th>àh,  HuVện  Yên  Thành7  T3nh Nghệ  An 
189/176, TiêuĐiều  Bà Vãi,+1` 28, pơn Thuỷ, Huyện Qăn  Bàn Tỉnh Lào  Ci 
14b/112,  fư_ng  Mè Chạy  Chọs,  Ấp  Hiềm Nghi, Khu  phố  1,9 Xã  Mỹ  Sơn, Ninh  |ơn, nh Ninh  Thuận 
65/61, Đường  Nong  Dỏng2Cao  Kiến, Loani Dọae, Khu phố 5, SYa, Huyện  Qu<g Điền, Thừa Thiên  Huế  
52/188, Zẩm  Cẩm  Còn Trinh)  N1, 29, -ã Co  Xá, Lâm  TV3ao, P&ú Thọ  
148/183,+Ém Lị Danho  Xóm 24S Khu  phố  !4N  VănULung, Phú  họm  Tỉnh  Phú Thọ  
5/8,  Tống  Biệ>  Ngầm, 18,  Khu  phố 1,H3uyện  Vạn  NDnh,  Tỉzh  K$ánh Hòa 
122/99',  Đường Phóng Dâu  Gia,  15, Khuphố  15,0Xã Quế  An,  9uYện Quế]Sơn, QYảng  N[m 
121/24, Chúa Nội  Ch0nh, Đội TiếnTới,PKh*  phố  23, Pa  Vệ Sử, MXq<g Tè,  Lai  +hâu 
5/62, Đường  Thành a*nh  Sắp  Đặt,  BiPKịcD, Khuphối5,  Phú  Thọ,  Phú  Thọ  
63-131,  ;à  Sà 0úp,  B,  Khu phố35,  Phường  Trường Lạc,gÔ Môn